# Analyzing CIA Factbook Data Using SQL

In [1]:
# Downloads ipython-sql module
!conda install -yc conda-forge ipython-sql

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/luc046/opt/anaconda3

  added / updated specs:
    - ipython-sql


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py38h50d1736_0         141 KB  conda-forge
    conda-4.10.3               |   py38h50d1736_0         3.1 MB  conda-forge
    ipython-sql-0.3.9          |py38h32f6830_1002          27 KB  conda-forge
    prettytable-2.1.0          |     pyhd8ed1ab_0          23 KB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    sqlparse-0.4.1             |     pyh9f0ad1d_0          34 KB  conda-forge
    ---------------------------------------------------------

In [7]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db
# This cell connects Jupyter Notebook to database file

In [8]:
%%sql
SELECT * 
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"
